In [1]:
import geopandas as gpd
import pygeos as pygeos

from geopy import distance
from sklearn.preprocessing import LabelEncoder

import pandas as pd
from simpledbf import Dbf5

/Users/rita/anaconda3/envs/altairpy37/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [2]:
def calc_dist(df):
    x, y = df['centroid_x'], df['centroid_y']
    
    x_nn, y_nn = df['nn_centroid_x'], df['nn_centroid_y']
    
    return distance.distance((y,x), (y_nn, x_nn)).meters

In [6]:
roads_shp = "portugal-latest-free.shp 2/gis_osm_roads_free_1.shp"

In [7]:
roads = gpd.read_file(roads_shp)

In [8]:
freedom_roads = roads[(roads.name.str.contains("25 de Abril")) | (roads.name.str.contains("Liberdade")) | (roads.name.str.contains("Salgueiro Maia")) | (roads.name.str.contains("Capitães de Abril"))]

In [9]:

df2 = freedom_roads[['osm_id', 'name', 'geometry']]
df2 = df2.rename(columns={'name': 'rua'})

In [10]:
df2.head(3)

,osm_id,rua,geometry
224,4453413,Avenida da Liberdade,"LINESTRING (-9.02960 39.75781, -9.02954 39.757..."
237,4472168,Avenida da Liberdade,"LINESTRING (-9.03221 39.75903, -9.03204 39.758..."
430,5044364,Via 25 de Abril,"LINESTRING (-16.90216 32.65268, -16.90207 32.6..."


In [11]:
concelhos = gpd.read_file('concelhos-shapefile/concelhos.shp', SHAPE_RESTORE_SHX='YES')

concelhos = concelhos.rename(columns={'NAME_2': 'concelho'})

df2['centroid_json'] = df2.geometry.apply(lambda x: x.centroid)
df2 = df2.sjoin(concelhos[['concelho', 'geometry']], how="left")

In [12]:
freguesias = gpd.read_file('cont-aad-caop2017/Cont_AAD_CAOP2017.shp', SHAPE_RESTORE_SHX='YES')

In [13]:
df2 = df2.drop(columns=['index_right'])

In [14]:
freguesias.to_crs(epsg=4326, inplace=True)

In [15]:
df2 = df2.sjoin(freguesias[['Freguesia', 'geometry']], how="left")

In [16]:
# Spatial join leading to some duplicates
df2 = df2.drop_duplicates('osm_id')

In [17]:
df2['centroid_x'] = df2.geometry.apply(lambda x: x.centroid.x)
df2['centroid_y'] = df2.geometry.apply(lambda x: x.centroid.y)

In [18]:
df2.Freguesia = df2.Freguesia.fillna('')

In [19]:
df2.head(3)

,osm_id,rua,geometry,centroid_json,concelho,index_right,Freguesia,centroid_x,centroid_y
224,4453413,Avenida da Liberdade,"LINESTRING (-9.02960 39.75781, -9.02954 39.757...",POINT (-9.02876 39.75742),Marinha Grande,1081.0,Marinha Grande,-9.028758,39.757420
237,4472168,Avenida da Liberdade,"LINESTRING (-9.03221 39.75903, -9.03204 39.758...",POINT (-9.03106 39.75850),Marinha Grande,1081.0,Marinha Grande,-9.031063,39.758498
430,5044364,Via 25 de Abril,"LINESTRING (-16.90216 32.65268, -16.90207 32.6...",POINT (-16.90205 32.65254),Funchal,NaN,,-16.902049,32.652540


In [20]:
# m = folium.Map(location=[39.9, -8.22])

#for i, row in df2.iterrows():
#    folium.Marker([row['centroid_y'], row['centroid_x']]).add_to(m)


In [21]:
df2['rua_freguesia_concelho'] = df2['rua'] + "_" +  df2['Freguesia'] + "_" + df2['concelho']
df2

,osm_id,rua,geometry,centroid_json,concelho,index_right,Freguesia,centroid_x,centroid_y,rua_freguesia_concelho
224,4453413,Avenida da Liberdade,"LINESTRING (-9.02960 39.75781, -9.02954 39.757...",POINT (-9.02876 39.75742),Marinha Grande,1081.0,Marinha Grande,-9.028758,39.757420,Avenida da Liberdade_Marinha Grande_Marinha Gr...
237,4472168,Avenida da Liberdade,"LINESTRING (-9.03221 39.75903, -9.03204 39.758...",POINT (-9.03106 39.75850),Marinha Grande,1081.0,Marinha Grande,-9.031063,39.758498,Avenida da Liberdade_Marinha Grande_Marinha Gr...
430,5044364,Via 25 de Abril,"LINESTRING (-16.90216 32.65268, -16.90207 32.6...",POINT (-16.90205 32.65254),Funchal,NaN,,-16.902049,32.652540,Via 25 de Abril__Funchal
436,5045625,Via 25 de Abril,"LINESTRING (-16.90216 32.65268, -16.90227 32.6...",POINT (-16.90228 32.65270),Funchal,NaN,,-16.902285,32.652700,Via 25 de Abril__Funchal
966,10073105,Ponte 25 de Abril,"LINESTRING (-9.17801 38.70838, -9.17807 38.708...",POINT (-9.17779 38.69398),Almada,608.0,"União das freguesias de Almada, Cova da Piedad...",-9.177787,38.693978,Ponte 25 de Abril_União das freguesias de Alma...
...,...,...,...,...,...,...,...,...,...,...
1164321,1044773608,Rua 25 de Abril,"LINESTRING (-9.09391 39.49901, -9.09379 39.49901)",POINT (-9.09385 39.49901),Alcobaça,1010.0,Alfeizerão,-9.093846,39.499012,Rua 25 de Abril_Alfeizerão_Alcobaça
1164322,1044773609,Rua 25 de Abril,"LINESTRING (-9.09379 39.49901, -9.09355 39.499...",POINT (-9.09332 39.49899),Alcobaça,1010.0,Alfeizerão,-9.093324,39.498986,Rua 25 de Abril_Alfeizerão_Alcobaça
1164323,1044773610,Rua 25 de Abril,"LINESTRING (-9.09457 39.49901, -9.09442 39.499...",POINT (-9.09424 39.49901),Alcobaça,1010.0,Alfeizerão,-9.094240,39.499008,Rua 25 de Abril_Alfeizerão_Alcobaça
1164326,1044773613,Rua 25 de Abril,"LINESTRING (-9.09494 39.49905, -9.09457 39.49901)",POINT (-9.09476 39.49903),Alcobaça,1010.0,Alfeizerão,-9.094758,39.499029,Rua 25 de Abril_Alfeizerão_Alcobaça


In [32]:
le = LabelEncoder()
df2['unique_index'] = le.fit_transform(df2.rua_freguesia_concelho)
df2 = df2.drop(columns=['centroid_json'])
df2

,osm_id,rua,geometry,concelho,Freguesia,centroid_x,centroid_y,rua_freguesia_concelho,unique_index
224,4453413,Avenida da Liberdade,"LINESTRING (-9.02960 39.75781, -9.02954 39.757...",Marinha Grande,Marinha Grande,-9.028758,39.757420,Avenida da Liberdade_Marinha Grande_Marinha Gr...,167
237,4472168,Avenida da Liberdade,"LINESTRING (-9.03221 39.75903, -9.03204 39.758...",Marinha Grande,Marinha Grande,-9.031063,39.758498,Avenida da Liberdade_Marinha Grande_Marinha Gr...,167
430,5044364,Via 25 de Abril,"LINESTRING (-16.90216 32.65268, -16.90207 32.6...",Funchal,,-16.902049,32.652540,Via 25 de Abril__Funchal,1378
436,5045625,Via 25 de Abril,"LINESTRING (-16.90216 32.65268, -16.90227 32.6...",Funchal,,-16.902285,32.652700,Via 25 de Abril__Funchal,1378
966,10073105,Ponte 25 de Abril,"LINESTRING (-9.17801 38.70838, -9.17807 38.708...",Almada,"União das freguesias de Almada, Cova da Piedad...",-9.177787,38.693978,Ponte 25 de Abril_União das freguesias de Alma...,338
...,...,...,...,...,...,...,...,...,...
1164321,1044773608,Rua 25 de Abril,"LINESTRING (-9.09391 39.49901, -9.09379 39.49901)",Alcobaça,Alfeizerão,-9.093846,39.499012,Rua 25 de Abril_Alfeizerão_Alcobaça,436
1164322,1044773609,Rua 25 de Abril,"LINESTRING (-9.09379 39.49901, -9.09355 39.499...",Alcobaça,Alfeizerão,-9.093324,39.498986,Rua 25 de Abril_Alfeizerão_Alcobaça,436
1164323,1044773610,Rua 25 de Abril,"LINESTRING (-9.09457 39.49901, -9.09442 39.499...",Alcobaça,Alfeizerão,-9.094240,39.499008,Rua 25 de Abril_Alfeizerão_Alcobaça,436
1164326,1044773613,Rua 25 de Abril,"LINESTRING (-9.09494 39.49905, -9.09457 39.49901)",Alcobaça,Alfeizerão,-9.094758,39.499029,Rua 25 de Abril_Alfeizerão_Alcobaça,436


In [33]:
df2.to_file('ruas_abril.geojson', driver='GeoJSON')